# Script for Figure 5d

In this script, a fully expanded C3 source leaf with plastidic ATP shuttles and malate valve constrained based on published studies is modelled in a scenario when photon uptake not forced on the system. Fluxes are observed for Figure 5d.

In [1]:
#import functions from library
from libsbml import readSBML
from cobra import io,flux_analysis
import re
from cobra.core import Metabolite, Reaction
from studyFunctions import *

#import model. Update file name and location in the next line
cobra_model = io.sbml.create_cobra_model_from_sbml_file("core_model.xml")

#set up a diel model, allow for day-night metabolite accumulations and constrain model to represent C3 leaf diel metabolism
cobra_model=setupDielModel(cobra_model,"MetabolitesToTransfer.txt")

0.7 MAL_v1 + 0.3 aMAL_v1 <=> 0.7 MAL_v2 + 0.3 aMAL_v2
0.5 CIT_v1 + 0.5 aCIT_v1 <=> 0.5 CIT_v2 + 0.5 aCIT_v2
bHIS_v1 --> bHIS_v2


##### Increase upper and lower bounds to 2000 and -2000 respectively in order to allow for PPFD upto 1500

In [2]:
for rxn in cobra_model.reactions:
    if rxn.lower_bound < -999:
        rxn.lower_bound = -2000
    if rxn.upper_bound > 999:
        rxn.upper_bound = 2000

##### Constrain plastidic malate valve based on published data

In [3]:
#Plastidic malate valve was constrained based on Fridlyand LE, Backhausen JE, Scheibe R (1998) Flux control of the malate valve in leaf cells. 349: 290–298
cobra_model.reactions.get_by_id("OAA_MAL_pc1").lower_bound = 0.75
cobra_model.reactions.get_by_id("OAA_MAL_pc1").upper_bound = -0.75

##### Constrain chlorplast malate valve based on published data

In [4]:
#For PEP-Pyr shuttle, constrain cplast PPDK based on Ishimaru, K., Ichikawa, H., Matsuoka, M., & Ohsugi, R. (1997). Analysis of a C4 maize pyruvate, orthophosphate dikinase expressed in C3 transgenic Arabidopsis plants. Plant Science, 129(1), 57-64.
cobra_model.reactions.get_by_id("PYRUVATEORTHOPHOSPHATE_DIKINASE_RXN_p1").lower_bound = 0
cobra_model.reactions.get_by_id("PYRUVATEORTHOPHOSPHATE_DIKINASE_RXN_p1").upper_bound = 0.03375
#For TP-3PGA shuttle, constrain cytosolic phosphorylating GAPDH activity based on Gibon, Y., et al(2004). A robot-based platform to measure multiple enzyme activities in Arabidopsis using a set of cycling assays: comparison of changes of enzyme activities and transcript levels during diurnal cycles and in prolonged darkness. The Plant Cell, 16(12), 3304-3325.
cobra_model.reactions.get_by_id("GAPOXNPHOSPHN_RXN_c1").lower_bound = -95.387597
cobra_model.reactions.get_by_id("GAPOXNPHOSPHN_RXN_c1").upper_bound = 95.387597
#and constrain cytosolic non-phosphorylating GAPDH based on Strand, Å., Zrenner, R., Trevanion, S., Stitt, M., Gustafsson, P., & Gardeström, P. (2000). Decreased expression of two key enzymes in the sucrose biosynthesis pathway, cytosolic fructose‐1, 6‐bisphosphatase and sucrose phosphate synthase, has remarkably different consequences for photosynthetic carbon metabolism in transgenic Arabidopsis thaliana. The Plant Journal, 23(6), 759-770.
cobra_model.reactions.get_by_id("1_PERIOD_2_PERIOD_1_PERIOD_9_RXN_c1").lower_bound = 0
cobra_model.reactions.get_by_id("1_PERIOD_2_PERIOD_1_PERIOD_9_RXN_c1").upper_bound = 0.33

In [5]:
phloem_model = cobra_model.copy()

### Constrain the model output so that net CO2 uptake matches experimental data for the chosen light input value (200 PPFD)


    Data extracted Fig 3 of Donahue et al 1997, see Script1.py.

In [7]:
"""
x = light, y = net CO2 uptake
y = a + bx + c^2"""
a = 0.0871351015562
b = 0.0291441670197
c = -0.000009013134

light=list()
CO2=list()
for x in range(100,1550,50):
    light.append(x)
    y=((a)+(b*x)+(c*(x**2)))
    CO2.append(y)
    
light_CO2=dict(zip(light, CO2))

import pandas as pd
df = pd.DataFrame(data={"PPFD":light,"Net CO2 uptake":CO2})
df = df[["PPFD","Net CO2 uptake"]]

    Model flux distribution based on assimilation rate based on PPFD data; and write flux distributions and FVA to file for figure 5 d

In [9]:
WTphloemOut = dict()
solutiondict_phloem = dict()
FVAdict_phloem = dict()
PPFD_list=list()


PPFD = 200
tempModel3 = phloem_model.copy()
tempModel3.reactions.get_by_id("Photon_tx1").upper_bound = PPFD*0.9
#tempModel3.reactions.get_by_id("Photon_tx1").lower_bound = PPFD*0.9
tempModel3.reactions.get_by_id("ATPase_tx1").upper_bound = estimateMaintenance(200)
tempModel3.reactions.get_by_id("ATPase_tx1").lower_bound = estimateMaintenance(200)
solution=flux_analysis.parsimonious.pfba(tempModel3)
WTphloemOut[PPFD] = estimateOutputFromNetCO2(tempModel3,light_CO2[PPFD],verbose=False)
tempModel3.reactions.get_by_id("diel_biomass").upper_bound = round(WTphloemOut[PPFD],3)
tempModel3.reactions.get_by_id("diel_biomass").lower_bound = round(WTphloemOut[PPFD],3)
solution=flux_analysis.parsimonious.pfba(tempModel3)
solutiondict_phloem[PPFD]=solution.x_dict

fva_result = flux_analysis.flux_variability_analysis(tempModel3,fraction_of_optimum=1)
FVAdict_phloem[PPFD] = fva_result

In [10]:
printFluxesAndFVA(tempModel3,solutiondict_phloem,FVAdict_phloem,outfile="Fluxes_Source_Leaf_ATPandMALshuttlesconstrained.csv")